In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter

import dsds.transform as t
import dsds.fs as fs

from sklearn.preprocessing import power_transform
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [2]:
orig_x, orig_y = make_classification(n_samples = 200_000, n_features = 100, n_informative = 60, n_redundant = 40)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,18.838365,-3.138532,-3.873528,17.070415,-13.560525,1.96571,-1.965082,5.783469,0.428268,-38.797151,8.298764,-12.576353,-1.861633,2.773823,-12.238886,-3.199177,6.614998,-2.187336,-0.391769,4.380679,-5.816873,28.007828,3.035804,1.063445,-3.112358,5.286622,1.060309,-10.994367,-24.714711,-3.657667,-0.994755,-9.641785,15.143455,-18.485327,7.833723,2.493201,…,-3.325263,-0.152426,9.123051,-5.145718,-9.061036,-21.139731,-3.164577,-1.871301,0.452645,9.04986,-17.856703,-1.658015,8.837662,2.566032,3.868969,13.603604,0.686215,-7.504418,4.868321,-1.68073,-1.107347,-1.622617,18.155519,-3.224936,0.548054,5.286467,-3.436789,-7.166324,2.650144,28.521399,-11.887173,5.832383,-23.413854,25.778616,1.579484,1.506464,23.302198
1,-45.019365,51.049535,-5.147741,45.127501,-14.515643,-5.018504,2.552445,3.455494,6.597109,35.936024,-5.255318,3.898475,3.778818,-2.044775,13.88813,1.409741,-4.689749,-0.443432,0.775308,16.732096,-5.367917,-34.597867,8.835069,-40.865362,3.531179,-5.986495,-6.15206,-10.979903,-23.250239,22.385368,0.39762,2.274617,-38.234934,13.165706,0.289677,6.131736,…,-4.440772,0.629373,-11.66193,7.177214,-46.468968,12.112405,6.161381,-9.49249,3.453204,2.911913,9.461024,3.836779,19.58101,-3.433667,-13.186078,29.013865,11.677664,2.87304,-4.061476,9.920809,5.670573,8.196527,-1.907671,7.275023,6.795811,-2.733096,-0.667286,-6.087519,5.769974,31.460689,1.997312,2.772209,0.487488,16.414308,0.792826,5.118932,-18.372568
0,23.582107,25.078874,-0.480541,-0.057588,-7.837307,-0.81084,0.710464,-1.435622,-1.325621,19.4166,1.417901,-15.392447,4.042859,-9.797768,28.419735,0.848428,-0.671895,-3.812379,8.478334,13.05808,-3.858833,23.997921,-1.425522,8.852922,-1.811088,-8.484258,-6.416268,23.650012,1.426985,-8.229819,0.861158,0.723967,-23.681811,-7.145042,-0.735262,6.576272,…,7.102614,6.448761,25.71455,-7.870966,-15.81299,39.278648,-1.173909,9.445686,4.722315,-4.889691,16.844918,5.412903,-6.761988,-3.286036,-2.14258,-14.286601,-3.950656,-0.007495,1.311505,2.704642,-6.610374,-2.103818,-7.4272,4.691626,-4.130149,9.108433,-2.747037,-1.559376,-14.807781,-28.097378,26.485988,21.976693,-2.948409,-4.954285,2.116492,-0.93181,0.811181
1,-59.681149,16.955993,-0.227823,-13.398623,-6.723666,6.927926,6.086259,-3.254116,5.810667,-34.192574,3.077547,-4.362404,5.866433,-5.79779,-15.128465,0.846538,-0.889667,-2.156517,-2.040406,1.668983,-5.844555,14.665437,3.177793,-35.094297,2.141413,1.121693,-13.229613,18.304359,-17.588345,5.401783,-4.320785,7.983602,-5.019301,14.836687,4.114632,6.019827,…,20.08193,-11.832234,-1.727852,49.206854,-58.781845,26.109764,2.754116,1.413501,-5.400721,4.008118,-26.994811,5.608762,-9.708903,3.631791,5.440808,-28.211472,2.043276,1.720479,3.652225,-2.108952,6.956553,1.076652,35.674569,-3.19722,-2.479417,3.830718,4.413153,1.873871,-41.679222,13.992926,2.354622,-11.589698,13.91484,-4.375771,-0.103554,-2.034978,18.760649
1,-26.752178,-14.823673,-0.355871,-6.058942,-38.216031,0.760833,10.535521,-7.500467,-1.620028,-70.126303,-8.996078,-26.302375,-11

## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Fscore
2. Mutual Information Score
3. MRMR feature selection strategies
4. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Fscore

In [4]:
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.03s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",4.810007,0.028296
"""column_1""",3430.169704,0.0
"""column_2""",0.032048,0.857922
"""column_3""",923.478699,2.2411e-202
"""column_4""",225.678368,5.5684e-51
"""column_5""",2.239449,0.134531
"""column_6""",2506.381939,0.0
"""column_7""",2453.692294,0.0
"""column_8""",9585.77082,0.0


In [5]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.33s in computing Fscore.


,feature,f_value,p_value
0,column_0,4.810007,2.829608e-02
1,column_1,3430.169704,0.000000e+00
2,column_2,0.032048,8.579219e-01
3,column_3,923.478699,2.241069e-202
4,column_4,225.678368,5.568382e-51
5,column_5,2.239449,1.345305e-01
6,column_6,2506.381939,0.000000e+00
7,column_7,2453.692294,0.000000e+00
8,column_8,9585.770820,0.000000e+00
9,column_9,425.186219,2.275543e-94


In [6]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.10s in computing Fscore.


,feature,f_value,p_value
0,column_0,4.810007,2.829608e-02
1,column_1,3430.169704,0.000000e+00
2,column_2,0.032048,8.579219e-01
3,column_3,923.478699,2.241069e-202
4,column_4,225.678368,5.568382e-51
5,column_5,2.239449,1.345305e-01
6,column_6,2506.381939,0.000000e+00
7,column_7,2453.692294,0.000000e+00
8,column_8,9585.770820,0.000000e+00
9,column_9,425.186219,2.275543e-94


# Mutual Information Score

In [7]:
# Took 16s
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 100/100 [00:16<00:00,  6.12it/s]


feature,estimated_mi
str,f64
"""column_94""",0.030536
"""column_61""",0.027874
"""column_36""",0.025165
"""column_8""",0.024947
"""column_39""",0.024708
"""column_38""",0.024587
"""column_40""",0.024553
"""column_31""",0.024136
"""column_46""",0.023763


In [8]:
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [9]:
# Took 1m38s seconds by using sklearn. The reason is that sklearn's implementation did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality.
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_94""",0.030536
"""column_61""",0.027874
"""column_36""",0.025166
"""column_8""",0.024944
"""column_39""",0.024707
"""column_38""",0.024583
"""column_40""",0.024554
"""column_31""",0.024136
"""column_46""",0.023762


# MRMR Feature selection Strategy

In [10]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [11]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [12]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 50/50 [00:16<00:00,  3.02it/s]

Spent 20.32s to compute mrmr.


['column_94',
 'column_46',
 'column_40',
 'column_71',
 'column_8',
 'column_39',
 'column_31',
 'column_38',
 'column_15',
 'column_70',
 'column_36',
 'column_61',
 'column_7',
 'column_55',
 'column_42',
 'column_11',
 'column_69',
 'column_22',
 'column_48',
 'column_44',
 'column_14',
 'column_98',
 'column_90',
 'column_6',
 'column_43',
 'column_10',
 'column_84',
 'column_96',
 'column_45',
 'column_82',
 'column_87',
 'column_51',
 'column_25',
 'column_34',
 'column_79',
 'column_81',
 'column_74',
 'column_41',
 'column_64',
 'column_88',
 'column_60',
 'column_89',
 'column_54',
 'column_29',
 'column_78',
 'column_27',
 'column_80',
 'column_67',
 'column_57',
 'column_1']

In [13]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:00<00:00, 115.18it/s]

Output is sorted in order of selection (relevance).
Spent 0.54s in computing.


['column_94',
 'column_46',
 'column_40',
 'column_71',
 'column_8',
 'column_39',
 'column_31',
 'column_38',
 'column_15',
 'column_70',
 'column_36',
 'column_61',
 'column_7',
 'column_55',
 'column_42',
 'column_11',
 'column_69',
 'column_22',
 'column_48',
 'column_44',
 'column_14',
 'column_98',
 'column_90',
 'column_6',
 'column_43',
 'column_10',
 'column_84',
 'column_96',
 'column_45',
 'column_82',
 'column_87',
 'column_51',
 'column_25',
 'column_34',
 'column_79',
 'column_81',
 'column_74',
 'column_41',
 'column_64',
 'column_88',
 'column_60',
 'column_89',
 'column_54',
 'column_29',
 'column_78',
 'column_27',
 'column_80',
 'column_67',
 'column_57',
 'column_1']

In [14]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:02<00:00, 24.99it/s]

Output is sorted in order of selection (relevance).
Spent 2.03s in computing.


['column_94',
 'column_46',
 'column_40',
 'column_71',
 'column_8',
 'column_39',
 'column_31',
 'column_38',
 'column_15',
 'column_70',
 'column_36',
 'column_61',
 'column_7',
 'column_55',
 'column_42',
 'column_11',
 'column_69',
 'column_22',
 'column_48',
 'column_44',
 'column_14',
 'column_98',
 'column_90',
 'column_6',
 'column_43',
 'column_10',
 'column_84',
 'column_96',
 'column_45',
 'column_82',
 'column_87',
 'column_51',
 'column_25',
 'column_34',
 'column_79',
 'column_81',
 'column_74',
 'column_41',
 'column_64',
 'column_88',
 'column_60',
 'column_89',
 'column_54',
 'column_29',
 'column_78',
 'column_27',
 'column_80',
 'column_67',
 'column_57',
 'column_1']

# Power Transform

In [15]:
# Eager transform.
start = perf_counter()
res_eager = t.power_transform(df, target = target, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Inferring best paramters: 100%|██████████| 100/100 [00:05<00:00, 18.64it/s]


Spent 5.64s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,18.876331,-3.145647,-3.877695,17.004304,-13.60708,1.967271,-1.94808,5.900669,0.429042,-39.549047,8.310745,-12.555044,-1.873156,2.774475,-12.311829,-3.203695,6.615674,-2.189387,-0.391803,4.36954,-5.806675,27.990488,3.047539,1.063183,-3.110399,5.265758,1.065951,-11.000945,-24.968882,-3.644689,-0.989882,-9.686514,15.245898,-18.431443,7.981127,2.521964,…,-3.347096,-0.152287,9.135027,-5.183596,-8.890677,-21.295491,-3.17327,-1.874043,0.452725,8.963499,-17.913387,-1.65984,8.805474,2.57389,3.86947,13.807792,0.68509,-7.547908,4.883575,-1.683127,-1.107991,-1.621245,18.307052,-3.224317,0.549559,5.248872,-3.461924,-7.08996,2.65206,28.07869,-11.887247,5.75152,-23.546565,26.188898,1.568098,1.520836,23.051457
1,-44.896488,50.650088,-5.154263,44.880737,-14.566939,-5.011335,2.579125,3.50744,6.681959,35.270707,-5.249341,3.902106,3.742043,-2.044384,13.801305,1.408608,-4.689351,-0.443549,0.775186,16.650558,-5.358907,-34.620919,8.896886,-40.935548,3.533584,-6.011885,-6.049286,-10.986469,-23.484007,22.576533,0.398519,2.269997,-37.874166,13.199174,0.290168,6.253061,…,-4.475446,0.631459,-11.644816,7.114774,-44.838029,12.04189,6.13663,-9.528482,3.456056,2.897073,9.438287,3.829442,19.480508,-3.421064,-13.182891,29.599359,11.55979,2.863453,-4.05005,9.88063,5.660878,8.215048,-1.902981,7.277232,6.904535,-2.746285,-0.668747,-6.027886,5.776601,30.955555,1.997308,2.747364,0.487238,16.633763,0.789478,5.227228,-18.555157
0,23.633882,24.923867,-0.48065,-0.057591,-7.858198,-0.810512,0.713293,-1.423901,-1.319546,19.127956,1.418591,-15.36403,4.001912,-9.793143,28.185514,0.847969,-0.671881,-3.817481,8.471914,13.000698,-3.853469,23.983869,-1.422262,8.844615,-1.810295,-8.527241,-6.306706,23.630666,1.423872,-8.184431,0.864937,0.723349,-23.49249,-7.131436,-0.732443,6.711307,…,7.031835,6.556405,25.766497,-7.943426,-15.433583,38.922034,-1.175517,9.410108,4.727006,-4.923937,16.792862,5.400306,-6.783631,-3.274297,-2.14239,-14.071316,-3.973658,-0.007495,1.313232,2.699308,-6.622657,-2.10168,-7.385954,4.692752,-4.080712,9.022945,-2.764472,-1.552883,-14.781209,-28.538678,26.485759,21.425387,-2.954412,-4.917412,2.097736,-0.925734,0.809952
1,-59.503724,16.865743,-0.227849,-13.445738,-6.74025,6.939705,6.193208,-3.207795,5.88053,-34.827854,3.080102,-4.35807,5.793286,-5.795685,-15.22721,0.846082,-0.889643,-2.15852,-2.041076,1.666715,-5.834282,14.658359,3.190435,-35.151636,2.142468,1.120113,-12.912158,18.290782,-17.747092,5.42588,-4.26282,7.94995,-4.999508,14.876368,4.169152,6.137769,…,19.765373,-11.573412,-1.727008,48.291595,-56.569391,25.903488,2.747213,1.411815,-5.394946,3.983277,-27.094971,5.595454,-9.746033,3.645646,5.441663,-27.659866,2.035532,1.716425,3.661901,-2.112483,6.943311,1.077321,36.055511,-3.196609,-2.457796,3.808054,4.376052,1.882825,-41.570025,13.827702,2.354616,-11.818994,13.85105,-4.345414,-0.103623,-2.011217,18.574721
1,-26.690837,-14.898773,-0.355933,-6.073511,-38.409587,0.761125,10.778861,-7.330786,-1.611417,-71.755696,-8.982598,-26.242684,-11.264606,

In [16]:
# Sklearn with Pandas

# In this case, the benefits of using dsds are:
# 1. Much better performance.
# 2. Users need to do extra steps to put data back in dataframe format.

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 18.05s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,0,18.876331,-3.145647,-3.877695,17.004304,-13.607080,1.967271,-1.948080,5.900669,0.429042,...,-7.089960,2.652060,28.078690,-11.887247,5.751520,-23.546565,26.188898,1.568098,1.520836,23.051457
1,1,-44.896488,50.650088,-5.154263,44.880737,-14.566939,-5.011335,2.579125,3.507440,6.681959,...,-6.027886,5.776601,30.955555,1.997308,2.747364,0.487238,16.633763,0.789478,5.227228,-18.555157
2,0,23.633882,24.923867,-0.480650,-0.057591,-7.858198,-0.810512,0.713293,-1.423901,-1.319546,...,-1.552883,-14.781209,-28.538678,26.485759,21.425387,-2.954412,-4.917412,2.097736,-0.925734,0.809952
3,1,-59.503724,16.865743,-0.227849,-13.445738,-6.740250,6.939705,6.193208,-3.207795,5.880530,...,1.882825,-41.570025,13.827702,2.354616,-11.818994,13.851050,-4.345414,-0.103623,-2.011217,18.574721
4,1,-26.690837,-14.898773,-0.355933,-6.073511,-38.409587,0.761125,10.778861,-7.330786,-1.611417,...,7.890067,-43.739408,-14.091316,-37.779883,-2.923238,18.055610,-18.730161,8.604254,-10.151824,-0.257237


# GC

In [17]:
import gc 
gc.collect()

304